## MLP for Handwritten Digit Classification

Objective: Implement a multi-layer perceptron (MLP) to classify handwritten digits from the scikit-learn Digits dataset. Evaluate how different activation functions affect early learning speed, convergence, and final test accuracy.

## 1. Data Loading and Preprocessing

The dataset is loaded from scikit-learn and preprocessed by:
- Normalising the input features
- Splitting into training and test sets (80-20)
- Wrapping the data in PyTorch DataLoaders for batching.

This ensures the MLP receives data in a format suitable for training.

In [3]:
import numpy as np
import torch

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset

# Data Loading and Preprocessing

# Load the Digits dataset
digits = load_digits()
X = digits.data
y = digits.target

# Set a random seed for reproducibility
torch.manual_seed(231)
np.random.seed(231)

# Split the data into training and test sets (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=231)

# Normalise the images by scaling pixel values to 0-1 range
normaliser = MinMaxScaler()
# Normaliser is fitted to the training data
X_train_normalised = normaliser.fit_transform(X_train) 
# Normaliser fit to training set applied to test set
X_test_normalised = normaliser.transform(X_test)

"""
Converting NumPy arrays into PyTorch tensors to allow for multi-dimensional data rep. & efficient computation
Using .float() for features since NN computations are more precise with floats.
"""
X_train_tensor = torch.from_numpy(X_train_normalised).float()
# .long() for target values, since Pytorch loss expects 'long' format
y_train_tensor = torch.from_numpy(y_train).long()
X_test_tensor = torch.from_numpy(X_test_normalised).float()
y_test_tensor = torch.from_numpy(y_test).long()

# Create DataLoader objects for both training and test sets
"""
Batch size 64 provides enough variability to aid generalisation,
and enough samples to calculate stable gradient estimates for smoother training.
"""
batch_size = 64

# TensorDataset() bundles multiple tensors into a single dataset
training_dataset = TensorDataset(X_train_tensor, y_train_tensor)
# DataLoader divides the dataset into smaller batches to allow for more frequent weight updates (once per batch).
# For training data, shuffle=true to improve generalisation by preventing the model from learning artifical order-related data patterns.
training_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
# For test data, shuffle=false to prevent introduction of randomness that isn't reflective of how the model would be used in real-world scenarios.
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## 2. Model Model Architecture

A simple multilayer perceptron is implemented with an input layer, one hidden layer, and an output layer. It accepts an activation function as a parameter to enable later experimentation.

In [5]:
# MLP Model Implementation

"""
Defining a neural network class by extending torch.nn.Module.
NN has one input layer with 64 neurons (since Digits dataset is 8x8 images), one hidden layer of 128 neurons, 
and one output layer with 10 neurons (since there are ten possible digit options).
It accepts an activation function as a parameter.
"""

# Set a random seed for reproducibility
torch.manual_seed(231)
np.random.seed(231)

import torch.nn.functional as F

# NeuralNetwork inherits from torch.nn.Module
class NeuralNetwork(torch.nn.Module):
  def __init__(self, activation_function):
    super().__init__() # super() calls the torch.nn.Module constructor
    # Define the three layers using torch.nn.Linear (creates fully connected/linear layers)
    # Input layer has 64 neurons (64 features) and transforms that input data into 128 neurons
    self.input_layer = torch.nn.Linear(64, 128)
    # Hidden layer with 128 neurons in and out
    self.hidden_layer = torch.nn.Linear(128, 128)
    # Output layer with 10 neurons out that correspond to the 10 classes (digits 0-9)
    self.output_layer = torch.nn.Linear(128, 10)
    self.activation_function = activation_function

  def forward(self, x):
    # Forward pass through the neural network
    x = self.activation_function(self.input_layer(x))
    x = self.activation_function(self.hidden_layer(x))
    softmax = torch.nn.Softmax(dim=1)
    x = softmax(self.output_layer(x))
    return x

## 3. Model Training

The `train_show` function trains the MLP for a set number of epochs using CrossEntropyLoss and the Adam optimiser. It prints the average training loss and accuracy per epoch.

In [7]:
# MLP Model Training 

# Hyperparameter tuning: Chose CrossEntropyLoss as loss function, decided on Adam optimiser over SGD, and set an appropriate learning rate. 

def train_show(network, lossFunc, optimiser, epochs):

    for epoch in range(epochs):
      # Storing every calculated loss and accuracy in the epoch to calculate average
      lossHistory = []
      accuracyHistory = []
      
      network.train() # Set model to training mode, which has active dropout to make the model more robust and prevent overfitting
        
      for data, targ in training_loader:
        optimiser.zero_grad() # PyTorch gradients accumulate by default, so they need to be zeroed out before each forward pass to avoid mixing between batches.

        y = network.forward(data) # Perform forward pass by calling forward() in NeuralNetwork class
        
        loss = lossFunc(y,targ) # Calculate the loss
        loss.backward() # Runs autograd to get the gradients needed by the optimiser: Computes gradients of the loss with respect to all model parameters using backpropagation.

        optimiser.step() # Takes a step: updates the model parameters in the direction that minimises the loss using the calculated gradients.

        """
        torch.argmax(y,dim=1) returns the index of the class with the highest
        probability for each input sample in the batch, which is then compared
        to the actual (targ) class value for that sample. A boolean tensor is
        returned to indicate whether the prediction is correct or not,
        and .float() converts that boolean tensor into a float tensor
        (1 for correct, 0 for incorrect).
        torch.mean() calculates the mean of all the input samples' float tensors
        to effectively get the proportion of correct predictions for that batch.
        """
        accuracy = torch.mean((torch.argmax(y,dim=1) == targ).float())
          
        # Add the loss and accuracy values to their lists. These lists will later be used to calculate the average over the epoch.
        lossHistory.append(loss.detach().item()) # Extracting the loss value as a Python scalar.
        accuracyHistory.append(accuracy.detach()) # Detaches the accuracy tensor from the computation graph

      # Calculate average loss and accuracy for the current epoch
      avg_loss = sum(lossHistory) / len(lossHistory)
      avg_accuracy = sum(accuracyHistory) / len(accuracyHistory)

      # Print average training loss and accuracy over the epoch
      print(f"Epoch {epoch+1}: Loss = {avg_loss:.3f}, Accuracy = {int(avg_accuracy*100)}%")

## 4. Model Evaluation

The `test_network` function evaluates a trained MLP on the test set and prints the final test accuracy.

In [9]:
# Model Evaluation on Test Set

# Measuring the model's accuracy on the test set.

# Set a random seed for reproducibility
torch.manual_seed(231)
np.random.seed(231)

def test_network(network):
  network.eval() # Set the network to evaluation mode, which disables dropout and normalises data using running mean and variance estimates collected during model training
  
  # Storing every calculated loss and accuracy to calculate the average
  lossHistory = []
  accuracyHistory = []
  
  with torch.no_grad(): # Disable gradient computation
    for test_data, test_targ in test_loader:
      y_test = network(test_data) # Perform forward pass
      test_loss = lossFunction(y_test, test_targ) # Compute the loss
      test_accuracy = torch.mean((torch.argmax(y_test,dim=1) == test_targ).float()) # Compute the accuracy

      # Add loss and accuracy to their lists to calculate the average later.
      lossHistory.append(test_loss.detach().item())
      accuracyHistory.append(test_accuracy.detach())

    # Calculate the average loss and accuracy for the test set.
    avg_loss = sum(lossHistory) / len(lossHistory)
    avg_accuracy = sum(accuracyHistory) / len(accuracyHistory)

    # Print the test accuracy
    print(f"Test Accuracy: {int(torch.round(avg_accuracy * 100).item())}%")

## 5. Activation Function Experiments
Experimenting with three different activation functions (ReLU, Sigmoid, Tanh) to assess:
- Early learning speed: How quickly the network improves accuracy in the first few epochs.
- Convergence behaviour: How smoothly training progresses over 15 epochs.
- Final test performance: Accuracy on the unseen test set.
This analysis highlights how activation choice affects both optimisation dynamics and generalisation.

In [11]:
# Experimenting with Activation Functions: Comparing the effectiveness of ReLU, Sigmoid, and Tanh.

"""
Activation functions introduce non-linearity into the model.

ReLU: (max(z, 0)), replaces any negative values with 0
and preserves positive values; activates only the neurons
with positive outputs to make the model sparse and efficient.

Sigmoid: Maps any input z to a value between 0 and 1, which can be
interpreted as probabilities.

Tanh: Maps any input z to a value between -1 and 1, makes it zero-centered
which can help center the data and make optimisation easier.

ReLU and Tanh often used in hidden layers, Sigmoid in output layers for binary classfication.
"""
activation_functions = {
    'ReLU': F.relu,
    'Sigmoid': torch.sigmoid,
    'Tanh': torch.tanh
}

# Train and test the network with each activation function
for name, act_fnc in activation_functions.items():
    print(f"\n{name} activation function:")
    # Instantiating the neural network using the previously defined NeuralNetwork class
    model = NeuralNetwork(act_fnc)
    # Defining the loss function
    lossFunction = torch.nn.CrossEntropyLoss() # calculates how off predictions are from actual values
    # Choosing an optimiser and an appropriate learning rate: chose Adam over SGD since Adam has adaptive learning rate for each parameter
    optimiser = torch.optim.Adam(model.parameters(), lr=0.001)
    # train_show will print all 15 epoch results
    print("Model Training:")
    train_show(model, lossFunction, optimiser, 15)
    # test_network will print average test accuracy
    test_network(model)


ReLU activation function:
Model Training:
Epoch 1: Loss = 2.288, Accuracy = 41%
Epoch 2: Loss = 2.166, Accuracy = 59%
Epoch 3: Loss = 1.888, Accuracy = 66%
Epoch 4: Loss = 1.733, Accuracy = 80%
Epoch 5: Loss = 1.664, Accuracy = 84%
Epoch 6: Loss = 1.631, Accuracy = 86%
Epoch 7: Loss = 1.580, Accuracy = 93%
Epoch 8: Loss = 1.559, Accuracy = 94%
Epoch 9: Loss = 1.544, Accuracy = 94%
Epoch 10: Loss = 1.531, Accuracy = 95%
Epoch 11: Loss = 1.522, Accuracy = 96%
Epoch 12: Loss = 1.518, Accuracy = 96%
Epoch 13: Loss = 1.510, Accuracy = 97%
Epoch 14: Loss = 1.512, Accuracy = 96%
Epoch 15: Loss = 1.502, Accuracy = 97%
Test Accuracy: 95%

Sigmoid activation function:
Model Training:
Epoch 1: Loss = 2.304, Accuracy = 9%
Epoch 2: Loss = 2.300, Accuracy = 11%
Epoch 3: Loss = 2.297, Accuracy = 11%
Epoch 4: Loss = 2.290, Accuracy = 11%
Epoch 5: Loss = 2.269, Accuracy = 15%
Epoch 6: Loss = 2.224, Accuracy = 26%
Epoch 7: Loss = 2.171, Accuracy = 35%
Epoch 8: Loss = 2.106, Accuracy = 39%
Epoch 9: Loss

## 6. Activation Function Comparison

- *ReLU*: Fast early learning (41% -> 80% in the first 4 epochs) with steady convergence. Final training accuracy 97%, test accuracy 95%.
- *Sigmoid*: Very slow early learning (<15% accuracy until epoch 5) due to vanishing gradients. Final training accuracy 66%, test accuracy 63%.
- *Tanh*: Rapid early learning (34% -> 89% in the first 4 epochs) with smooth convergence. Final training accuracy 98%, test accuracy 96%, slightly outperforming ReLU.
  
**Conclusion**: For this MLP on the Digits dataset, **Tanh and ReLU are both effective**, with Tanh having a slight edge. Sigmoid underperforms due to slow convergence caused by vanishing gradients.